# RandomProp

## imports; helper functions

In [1]:
import itertools as it
import numpy as np
import numpy.random as rdm
from abc import ABC, abstractmethod

In [ ]:
# Tree data structure: necessary if we want to go beyond FOL
# A tree is a dict of {int : Tree}

# convert n-length list of lists into n-level uniform tree
# (for FOL sentences only)
def tree(xranges):
  if not xranges:
    return {}
  else:
    Tree_ = tree(xranges[1:])
    return { x:Tree_ for x in xranges[0]}

## RandomProp class

In [ ]:
"""
The general class of "Random Propositions", i.e. propositions of the form
P(x1, ... xn) generated from some probability distribution as defined in the
self.calc() method. Subclass this class with your own custom self.calc().

[n is fixed for FOL sentences; for hyperarithmetic sentences the number of
variables depends on the values of the variables already given]
"""


class RandomProp(ABC):

    def __init__(self):
        self.values = {}  # dict of all values generated so far in an instance

    @abstractmethod
    def calc(self, *xs):
        pass

    # if P(x1, ... xn) is already in self.values, return that;
    # if not, generate it with self.calc(), add it to self.values and return that
    def get_or_set(self, *xs):
        # self.values[xs] = self.values.get(xs, self.calc(*xs)) # SLOW!!
        if xs not in self.values:
            self.values[xs] = self.calc(*xs)
        return self.values[xs]

    # array version of self.values
    @property
    def array(self):
        xmaxs = np.array(
            list(map(max, zip(*self.values.keys())))
        )  # size of array needed, e.g. [100, 100]
        array = np.full(xmaxs + 1, np.nan)  # initialize with nans
        for xs, v in self.values.items():  # populate from dict
            array[xs] = v
        return array

    # compute P on all given inputs (x1, ... xn) and return in array form
    # (useful for a quick visualization)
    def build(self, xss):
        for xs in xss:
            _ = self.get_or_set(*xs)
        return self.array

    # finite-window estimate of truth value of corresponding Sigma or Pi sentence
    def test(
        self,
        position,  # "Sigma" or "Pi"
        xtree,  # specify the finite window
        free_xs=[],  # the variables of P already set (aux var for recursion)
    ):

        if xtree == {}:
            val = self.get_or_set(*free_xs)
        elif position == "Sigma":
            val = any(
                [
                    self.test("Pi", subtree, free_xs=free_xs + [x])
                    for x, subtree in xtree.items()
                ]
            )
        elif position == "Pi":
            val = all(
                [
                    self.test("Sigma", subtree, free_xs=free_xs + [x])
                    for x, subtree in xtree.items()
                ]
            )
        return int(val)

    # # finite-window estimate of truth value of corresponding Sigma or Pi sentence
    # def test(self,
    #          position, # "Sigma" or "Pi"
    #          *xranges, # specify the finite window e.g. "[2, 4, 3], range(9)"
    #          free_xs = [] # the variables of P already set (aux var for recursion)
    #          ):
    #   if not xranges: # Delta_0 case
    #     val = self.get_or_set(*free_xs)
    #   elif position == "Sigma":
    #     val = any([self.test("Pi", *xranges[1:], free_xs = free_xs + [x]) for x in xranges[0]])
    #   elif position == "Pi":
    #     val = all([self.test("Sigma", *xranges[1:], free_xs = free_xs + [x]) for x in xranges[0]])
    #   else:
    #     raise TypeError("Type must be 'Sigma' or 'Pi'!")
    #   return int(val)

    # test(), but *xtree is decided by players (i.e. construction algorithms)
    def play(
        self,
        position,
        constructors,  # a pair of programs to generate the tree alternately
    ):
        pass

    # monte-carlo estimate of probability of Sigma or Pi sentence being true
    # this is a class method, as each trial is a separate instance
    @classmethod
    def tests(cls, position, xtree, num_trials):
        return (
            sum([cls().test(position, xtree) for i in range(num_trials)]) / num_trials
        )

In [ ]:
"""
Example of a RandomProp. Defined as:
- P(x, 0) ~ Bernoulli(1 / (x + 1) ** 2)
- P(x, y + 1) ~ Bernoulli(exp (- 1 / #[P(x, <= y)] ** 2))
where #[P(x, <= y)] is the count of 1s so far for the same x, or "count_left".

Every instance of P represents a separate generation of the random variables,
i.e. a sample from the distribution of P.
"""
class P(RandomProp):

  def calc(self, x, y):
    count_left = np.count_nonzero([self.get_or_set(x, y_) for y_ in range(y)])
    if y == 0:
      p = 1 / (x + 1) ** 2
    elif count_left == 0:
      p = 0
    else:
      p = np.exp(-count_left**-2)
    v = rdm.binomial(1, p)
    return v

# An instance of P
p = P()
arr = p.build(it.product(range(10), range(10)))
print("An instance of P:\n", arr)

# Monte Carlo estimate of the probability of Ex.Ay.P(x,y).
# One may show theoretically that the real probability is
# P(Ex, Ay, P(x, y)) = 1 - sinc(pi * exp(-pi^2/12)) ~ 0.288.
pr = P.tests("Sigma", tree((range(10), range(50))), num_trials = 500)
print("\nMonte Carlo estimate of probability of Ex.Ay.P(x,y):", pr)

An instance of P:
 [[1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

Monte Carlo estimate of probability of Ex.Ay.P(x,y): 0.282


# RL agent

(under construction)

In [ ]:
# !pip install tensorflow
# !pip install gym
# !pip install keras
# !pip install keras-rl2

#https://www.section.io/engineering-education/building-a-reinforcement-learning-environment-using-openai-gym/
#https://gym.openai.com/envs/CartPole-v0/
#https://gym.openai.com/envs/SpaceInvaders-v0/
#https://towardsdatascience.com/reinforcement-learning-with-python-part-1-creating-the-environment-dad6e0237d2d
#https://medium.com/applied-data-science/how-to-train-ai-agents-to-play-multiplayer-games-using-self-play-deep-reinforcement-learning-247d0b440717
#https://towardsdatascience.com/beginners-guide-to-custom-environments-in-openai-s-gym-989371673952

from gym import Env
from gym.spaces import Box, Discrete